In [1]:
import pysubs2
import re
from stardict import DictCsv
# from stardict import LemmaDB
from googletrans import Translator
import pandas
from difflib import SequenceMatcher 
import itertools
import string

# depend on https://github.com/skywind3000/ECDICT/

In [2]:
def tranlate(source):

    import requests
    import json
    
    url = "http://api.interpreter.caiyunai.com/v1/translator"
    
    #WARNING, this token is a test token for new developers, and it should be replaced by your token
    token = "3975l6lr5pcbvidl6jl2"
    
    
    payload = {
            "source" : source, 
            "trans_type" : "en2zh",
            "request_id" : "demo",
            "detect": True,
            }
    
    headers = {
            'content-type': "application/json",
            'x-authorization': "token " + token,
    }
    
    response = requests.request("POST", url, data=json.dumps(payload), headers=headers)

    return json.loads(response.text)['target']

In [3]:
def longestSubstring(str1,str2): 
     # 两个字符串最长公共字符串
     # initialize SequenceMatcher object with  
     # input string 
    seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
    match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
    if (match.size!=0): 
          return (str1[match.a: match.a + match.size])  
    else: 
          return ""

In [4]:
def get_trans(word_trans_from_dict, word_trans_from_translator, sentence_trans):
    # 句子中的单词含义, 如果没有公共的, 就返回查到的词
    match=longestSubstring(sentence_trans,word_trans_from_dict)
    if match=="":
        return word_trans_from_translator.replace(r"\n", "").translate(str.maketrans('', '', string.punctuation))
    else:
        return match.replace(r"\n", "")

In [5]:
def word_unknown(word_query):
    if not(word_query):
        return False
    # 是否认识?
    tag_check={'cet4':False, 'cet6':False, 'toelf':True ,"gre":True,'ielts':True}
    collins_threshold=2; collins_default=True
    bnc_threshold=5000; bnc_default=True
    frq_threshold=5000; frq_default=True
    
    # check tag
    chk1=True
    chk2=True
    for (key, value) in tag_check.items():
        if not(value):
            chk1 = chk1 and not((key in word_query['tag']) if word_query['tag'] else True) 
        else: 
            chk2 = chk2 or ((key in word_query['tag']) if word_query['tag'] else True)
    tag_chk = chk1 and chk2
    
    # check collins
    collins_chk = (word_query['collins']<=collins_threshold) if word_query['collins']>=0 else collins_default
    
    # check bnc
    bnc_chk=(word_query['bnc']>=bnc_threshold) if word_query['bnc']>0 else bnc_default
    
    # check frq
    frq_chk=(word_query['frq']>=frq_threshold) if word_query['bnc']>0 else frq_default

    return (tag_chk+collins_chk+bnc_chk+frq_chk) >=3

In [6]:
def add_trans_to_sentence(s, sdict, translator, filter_word=True):
    sentence=s.replace("\\N", " ").replace("\n", " ").translate(str.maketrans('', '', string.punctuation))
    words=sentence.split()
    words_to_trans={}
    for word in words:
        word_query=sdict.query(word) if sdict.query(word) else sdict.query('unknown')   
        if filter_word:
            if word_unknown(word_query):
                words_to_trans[word]=word_query['translation']
        else:
            words_to_trans[word]=word_query['translation']
    to_trans_list=[[sentence], words_to_trans.keys()]
    to_trans_list=list(itertools.chain(*to_trans_list))
    trans=tranlate(to_trans_list)
    sentence_trans=trans[0]
    word_with_trans={}
    for idx, word in enumerate(words_to_trans.keys()):
        word_trans=trans[idx+1]
        word_with_trans[word]=get_trans(words_to_trans[word], word_trans, sentence_trans)
    for (word, meaning) in word_with_trans.items():
        meaning=word+"("+meaning+")"
        s=s[0:s.find(word)]+meaning+s[(s.find(word)+len(word)):]
    return s

In [7]:
dict_filename="ecdict.csv"
lemma_filename='lemma.en.txt'
sub_filename="01sub.srt"
subs = pysubs2.load(sub_filename, encoding="utf-8")

In [8]:
sdict=DictCsv(dict_filename)
# lemma = LemmaDB()
# lemma.load(lemma_filename)
translator = Translator()

In [9]:
s="They even understand ophthalmology"
print(add_trans_to_sentence(s, sdict, translator,filter_word=False))

They(他们) even(甚至) understand(懂) ophthalmology(眼科学)


In [10]:
for line in subs:
    s=line.text
    line.text=add_trans_to_sentence(s, sdict, translator)
    print(line.text)

We have arrived(到达)\Nat the dawn of a new era.
We will face the unknown\Ntogether.
I don't know what it is,
but there's plenty of people\Nout hertheres(有)here's plenty of people\Nout here
trying(尝试) to figure it out.
You know I've got your backIve(伊夫)u know I've got your back.
Damn well better,\NI'm your captain.
We have to work together.
For the good of\Nthe new Belter(了) state.
Detective Miller\Ncrashed(崩溃) into Venus.
I'm just an investigator now.
I find things(事情).
Couple hundred billion\Nbrain cells(细胞) in your skull.
When I push a few trillion(万亿)\Nof them buttons(按钮)
in exactly the right way,\Nyou're talking(说话) to Miller.
Miller showed(显示) me things(事情).
A whole civilization.
They made(成) the station,\Nthis space.
Something killed them.\NThey tried(试) to stop it.
Burned(烧焦了) whole solar systems(系).
Only, it didn't work.
When we detonated(引爆) our bomb,
I believe\Nwe became(成为) the threat.
We taught(教) the station
that our ships(的)\Nare(的) fusion(聚变) bombs(炸弹).
We need to shut do

In [11]:
subs.save("my_subtitles_edited.srt")